In [1]:
def get_authenticated_bitport_obj(name='Jonas'):
    import sys
    sys.path.append('/Users/mjk/python/various')
    import Bitport_Auth_obj
    from lxml import html

    bitport_obj = Bitport_Auth_obj.Bitport('nonsensename')

    bitport_obj.auth()
    return bitport_obj

In [2]:
def make_soup(url):

    from bs4 import BeautifulSoup
      
    r = o.session.get(url) 

    s = BeautifulSoup(r.content, 'html.parser')
    
    return s



In [3]:
def is_soup(soupx):
        from bs4 import BeautifulSoup

        if isinstance(soupx, BeautifulSoup):
            return True
        else:
            return False
    

In [4]:
def get_next_page(soup):
    
    # If there are more than 15 items, there will be a 'Next Page' url link at the bottom of the page.
    # Get and return that link, else return False;
    
    if not is_soup(soup):
        return("ERROR: get_next_page requres one arg: soup!")
    
    debug = False
    
    if debug: print(type(soup))
    
    if debug: pprint(soup)
        
    next_page_link = soup.find('a', {'class': 'btn btn-normal btn-small paginator-show-more change-url'})

    if next_page_link is None:
        if debug: print('No next page found.')
        return ''
    else: 
        if debug: print('Found a next page....')

        return 'https://bitport.io/' + next_page_link['href']
    

In [5]:

    

def parse_bitport_folder(soupx):
    import re
    import sys
    
    if soupx == '':
        print('ERROR: got empty soup!')
    
    if not is_soup(soupx):
        print("ERROR: parse_bitport_folder requres one arg: soup!, instead got".format(type(soupx)))
        sys.exit(5) 



    trs = soupx.find_all('a', {'class': 'mobile-hidden'} )
    
    '''

    The above find_all mostly these 4 types of things; Download, Move, Delete, Mark.   We are interested in just 
    the unique string ( '75odf161c5' in below examples).  Since there is NO OTHER WAY TO GET THIS, we will just 
    focus on the 'Move' type of thing, grab them all, and manipulate them to get what we really want.
    What we realyl want is, e.g:  https://bitport.io/my-files/file/75odf161c5"

    
    Download;

    <a class="mobile-hidden" data-title="Download" href="/my-files/download/uhew3rvdgnne3mym13wefmjnh6m4y3at/75odf161c5">
    <span class="fa fa-cloud-download"></span>
    </a>


    Move;

    <a class="mobile-hidden" data-title="Move" href="/my-files/move-file/75odf161c5" rel="ajax-popup">
    <span class="fa fa-sign-out"></span>
    </a>


    Delete;

    <a class="mobile-hidden confirmation-popup" data-confirmation="Do you really want to delete the file Animaniacs - 
    S1E15 - Pinky And The Brain - Battle for the Planet.avi?" data-title="Delete" 
    href="/my-files/8yzfuvl00s?fileCode=75odf161c5&amp;do=deleteFile">
    <span class="fa fa-trash-o"></span>
    </a>


    Mark;
    <a class="mobile-hidden btn-checkbox" data-title="Mark" href="#">
    <span class="fa fa-square-o"></span>
    </a>
    '''


    file_all={}
    parent=''
    childfile=''
    debug = False
    for row in trs:
        hash_thing = {}
        if row.has_attr('data-title'):
            if debug: print("Informational: Encountered a data-title: ", row['data-title'])
            if row['data-title'] == 'Delete':
                del_href = row['href']
                # Now we have isolated the 'Delete' piece.  (its the best piece).  Parse out the 'parent' string 
                # and the 'file' string.
                # e.g. the '8yzfuvl00s', and the 75odf161c5, respectively;
                # delete me; parent = re.sub(r'^/my-files/(\w+)[?](.+)$', r'\1', del_href)
                mat = re.match(r'^/my-files/(?P<parent>\w+)[?]fileCode=(?P<childfile>\w+)[&].*do=deleteFile', del_href)
                if mat:
                    parent = mat.group('parent')
                    childfile = mat.group('childfile')
                    if debug: print('Successfully parsed out parent: ', mat.group('parent')) 
                    if debug: print('Successfully parsed out childfile: ', mat.group('childfile'))
                    # Now we want a string that looks like this: https://bitport.io/my-files/file/75odf161c5
                    # So we use 'childfile' for that;
                    childpath = 'https://bitport.io/my-files/file/' + childfile
                    # Get fn_text (the 'data-confirmation' attribute has human readable file description text.); 
                    fn_text = re.sub('Do you really want to delete the file ', '', row['data-confirmation'])
                    file_all[childfile] = {'parent': parent, 'childpath': childpath, 'fn_text': fn_text }
                else:
                    print("WARNING!: NO Match from regex against:\n",  del_href)

        else:
            if debug: print("Informational: Encountered something with NO data-title")
    return file_all
    
    


In [6]:

def get_all_folders_child_urls(url):

    '''

    Takes a url of a bitport folder (subfolder of the top-level my-files dir), and returns a dict of this format;

    {'06oih1wxtj': {'childpath': 'https://bitport.io/my-files/file/06oih1wxtj',
                    'fn_text': 'Pinky And The Brain - S3E32 E33 - The Tailor And '
                               'The Mice, Bah, Wilderness.avi?',
                    'parent': '8yzfuvl00s'},
     '16rdodtbgx': {'childpath': 'https://bitport.io/my-files/file/16rdodtbgx',
                    'fn_text': 'Pinky And The Brain - S3E23 - The Real Life.avi?',
                    'parent': '8yzfuvl00s'},

    }
    '''

    one_dict = {}
    all_dict = {}
    debug = True
    current_soup = make_soup(url)
    one_dict = parse_bitport_folder(current_soup)
    next_url = get_next_page(current_soup)

    while next_url != '':

        current_soup = make_soup(next_url)
        one_dict = parse_bitport_folder(current_soup)

        for xx in one_dict.keys():
            all_dict[xx] = one_dict[xx]
        if debug: print('get_all_folders_child_urls, getting: {}/{}'.format(len(one_dict), len(all_dict)))

        next_url = get_next_page(current_soup)
        
    return all_dict





In [7]:
def get_mp4_download_link_from_one_page(urlx):
    '''
    Take a url of the bitport page for a single videl, and return a dict with the download url/link to 
    the file(both sizes).
    Here, we find/parse a bit that looks like this.  Importantly, the .../2 is usually 2x the size of the .../1.
    Anyone calling this function will need to know that since they may wish to download the large or the small
    (or both!)
    source src="https://cf05.bitport.io/download/GCQyPTbzJVIdvQbYwbVyalP6rTW0Hlqk/2" type="video/mp4"
    source src="https://cf05.bitport.io/download/GCQyPTbzJVIdvQbYwbVyalP6rTW0Hlqk/1" type="video/mp4"
    '''
    ans = {}
    soupz = make_soup(urlx)
    cnt = 2
    for xx in soupz.find_all({'source': 'src'}):
        ans[cnt] = xx['src']
        cnt-=1
    return ans


In [10]:
'''
def get_all_mp4_download_links_for_one_folder(folder_url):
    '''
    Take the url of a single folder, e.g.; https://bitport.io/my-files/8yzfuvl00s, and return a list of 
    all the mp4 download links found on all of its childpages.
    '''
    my_list = []
    all_child_urls = get_all_folders_child_urls(folder_url)
    print("get_all_mp4_download_links_for_one_folder got a dict of length; {}".format(len(all_child_urls)))
    for itemx in all_child_urls:
        my_list.append(all_child_urls[itemx]['childpath'])
    return my_list
'''

IndentationError: unexpected indent (<ipython-input-10-92d59a4e92d3>, line 4)

In [11]:
'''
list1 = get_all_mp4_download_links_for_one_folder('https://bitport.io/my-files/8yzfuvl00s')
from pprint import pprint
pprint(list1)
'''

"\nlist1 = get_all_mp4_download_links_for_one_folder('https://bitport.io/my-files/8yzfuvl00s')\nfrom pprint import pprint\npprint(list1)\n"

In [8]:
def get_all_mp4_links(dictx):
    '''
    Takes "The Dict" (see: get_all_folders_child_urls) returns two dicts: 1 and 2 (small and large)
    Each of these dicts has key =  download_url and val = text_description. 
    '''
    #print(' ')
    #print('%' * 80)
    #from pprint import pprint
    #pprint(dictx)
    #print('&' * 80)

    mp4_links_size_1 = {}
    mp4_links_size_2 = {}

    for key,val in dictx.items():
        #print("\n\n{} = {}".format(key, val))
        if 'childpath' in val:
        
            url = val['childpath']
            txt = val['fn_text']

            print("Fetching download links for: ", url)
            dictx = get_mp4_download_link_from_one_page(url)

            if 1 in dictx:
                mp4_links_size_1[dictx[1]] = txt
            else:
                print('Interesting: ', url, 'doesnt have url lownload link 1')

            if 2 in dictx:
                mp4_links_size_2[dictx[2]] = txt
            else:
                print('Interesting: ', url, 'doesnt have url lownload link 2')

        else:
            print('BAD ', rec)

            
    print('get_all_mp4_links returning list1 ({}), and list2({})'.format(len(mp4_links_size_1), len(mp4_links_size_2)))

    return ( mp4_links_size_1, mp4_links_size_2 )

In [9]:
'''
thing = get_mp4_download_link_from_one_page('https://bitport.io/my-files/file/f1n7m7sq37')
from pprint import pprint
print(type(thing))
pprint(thing)
'''

"\nthing = get_mp4_download_link_from_one_page('https://bitport.io/my-files/file/f1n7m7sq37')\nfrom pprint import pprint\nprint(type(thing))\npprint(thing)\n"

In [10]:
def download_file(download_url, file_path):
    '''
    Take a url and filename, use the requests lib in stream mode to download the file.
    '''
    import re
    
    file_path = re.sub(r'[?]$', '', file_path)  # removing trailing '?'                 
    
    print('Downloading file {}'.format(file_path), end='')
    rr = o.session.get(download_url, stream=True) 

    with open(file_path, 'wb') as fd:
        for chunk in rr.iter_content(chunk_size=2**24):  # 2^24 = 16MB
            print('.', end='')
            fd.write(chunk)
            
    print('\nDone.')
        
        

In [15]:
'''
for k,v in d1.items():
    print('key-', k)
    download_file(k, v)
'''

"\nfor k,v in d1.items():\n    print('key-', k)\n    download_file(k, v)\n"

In [11]:
class BitportMyFiles(object):

    """Class to represent the files you have in the bitport.io cloud website.   Specifically, it parses the
       ./my-files page's short or 'grid' format as can be seen via;  https://bitport.io/my-files?type=grid&do=setView


   Attributes:
       soup: (a BeautifulSoup obj): the only required parameter.  We take the soup and return a dict of your files.
       hrefs: dict keyed on filename.  Each file will have a (relative) URL where to find it.
       files: list of the file names.
       sizes: dict keyed on filename, with values specifying size in megabytes.
       space: dict with keys like 'used', 'total', 'remaining' describing the total space you have on the site.

   Methods:
   """
    def __init__(self, my_soup):
        self.soup = my_soup
        self.hrefs = {}
        self.files = []
        self.sizes = {}
        self.space = {}
        self.blob = []

        table = self.soup.find('ul', {'class': 'list list-files columns-5'})
        table_rows = table.find_all('li', {'class': ' folder'})
        print('TABLE_ROWS type is', type(table_rows))

        cnt = 0
        for row in table_rows:
            if row['data-size'] != '0':    # only care about size > 0 ('data-size' is a str)
                torrent_file_name = row.a.h2.text
                torrent_file_href = row.a['href']
                torrent_file_size = float(row['data-size'])

                self.hrefs[torrent_file_name] = torrent_file_href
                self.sizes[torrent_file_name] = torrent_file_size
                self.files.append(torrent_file_name)
                
                
                cnt += 1
                self.blob.append( [ torrent_file_href, torrent_file_name, torrent_file_size] )


In [17]:
# Build a list of top level Bitport files.  This func populates the <obj>.files attribute;

def get_file_list(a_bitport_obj):
    from bs4 import BeautifulSoup
    from pprint import pprint
    fblob = []  # a list of dicts.  One list member for each file
    cnt = 0
    
    bitport_files_page = a_bitport_obj.session.get(a_bitport_obj.base_url + '/my-files')
    bitport_files_page_html = bitport_files_page.content
    print('\tFunction "get_file_list": Bitport.session.get got response:[{}]'.format(bitport_files_page))
    files_obj = BitportMyFiles(BeautifulSoup(bitport_files_page_html, 'html.parser'))
    a_bitport_obj.files = {'key1': 'val1', 'key2': 'val2'}
    a_bitport_obj.hrefs = files_obj.hrefs
    a_bitport_obj.sizes = files_obj.sizes
    a_bitport_obj.files = files_obj.files
    
    return fblob
    
    #print('Interesting stuff about object "{}":'.format(a_bitport_obj.name))
    #print('\tbase_url:\t{}'.format(a_bitport_obj.base_url))
    #print('\tauthenticated:\t{}'.format(a_bitport_obj.authenticated))
    #print('\tuser_token:\t{}'.format(a_bitport_obj.data_user_token))
    #print('\thidden_token:\t{}'.format(a_bitport_obj.hidden_token))
    #print('\tuser_agent:\t{}'.format(a_bitport_obj.user_agent))
    #print('\tusername:\t{}'.format(a_bitport_obj.username))
    #print('\tfiles:')
    #pprint(a_bitport_obj.files, width=80)
    #a_bitport_obj.get_file_list()
    #a_bitport_obj.get_recapit()    
    


In [12]:
o2 = get_authenticated_bitport_obj('Cinnamon')


	Function "auth": Bitport.session.get got response:[<Response [503]>]
(b'<!DOCTYPE HTML>\n<html lang="en-US">\n<head>\n  <meta charset="UTF-8" />\n  '
 b'<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n  '
 b'<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" />\n  <meta'
 b' name="robots" content="noindex, nofollow" />\n  <meta name="viewport" co'
 b'ntent="width=device-width, initial-scale=1, maximum-scale=1" />\n  <title'
 b'>Just a moment...</title>\n  <style type="text/css">\n    html, body {widt'
 b'h: 100%; height: 100%; margin: 0; padding: 0;}\n    body {background-colo'
 b'r: #ffffff; font-family: Helvetica, Arial, sans-serif; font-size: 100%;}'
 b'\n    h1 {font-size: 1.5em; color: #404040; text-align: center;}\n    p {f'
 b'ont-size: 1em; color: #404040; text-align: center; margin: 10px 0 0 0;}\n'
 b'    #spinner {margin: 0 auto 30px auto; display: block;}\n    .attributio'
 b'n {margin-top: 20px;}\n    @-webkit-keyframes bubbles { 33%: { -w

In [ ]:
get_file_list(o2)

In [ ]:
o2.hrefs

In [33]:
def talk_to_human(ob):
    cnt = 0
    for item in ob.hrefs:
        # print(cnt, item)
        print('\t{0:<5} {1:<50} {2:<50}'.format(cnt, item, ob.hrefs[item]))
        cnt += 1

In [34]:
talk_to_human(o2)

	0     Game Of Thrones S06 Complete Season 6...           /my-files/fbjxvlav3i                              
	1     Game of Thrones S07 E03 1080p Web...               /my-files/v2h1bdqdip                              
	2     VA Billboard Hot 100 Singles Chart 3...            /my-files/ghkzjun3gj                              
	3     Game of Thrones S07 E04 SCR x264 ...               /my-files/yc2tp7c4b1                              
	4     Game of Thrones S07 E02 720p HDRip...              /my-files/5e6a1jppwn                              
	5     Game Of Thrones S07 Episode 1,2 WEB...             /my-files/tfeqka6gdo                              
	6     Pinky and the Brain                                /my-files/8yzfuvl00s                              


In [19]:
fl = get_file_list(o2)
from pprint import pprint
pprint(fl)

	Function "get_file_list": Bitport.session.get got response:[<Response [200]>]
None


In [58]:
DICT = get_all_folders_child_urls('https://bitport.io/my-files/8yzfuvl00s')
(d1, d2) = get_all_mp4_links(DICT)
from pprint import pprint
pprint(ans)

get_all_folders_child_urls, getting: 15/15
get_all_folders_child_urls, getting: 15/30
get_all_folders_child_urls, getting: 15/45
get_all_folders_child_urls, getting: 10/55
Fetching download links for:  https://bitport.io/my-files/file/e4lawhx85t
Fetching download links for:  https://bitport.io/my-files/file/kx5kg0xm4r
Fetching download links for:  https://bitport.io/my-files/file/c4xhlmjcsu
Fetching download links for:  https://bitport.io/my-files/file/n17umcud4f
Fetching download links for:  https://bitport.io/my-files/file/41fuwbbe7p
Fetching download links for:  https://bitport.io/my-files/file/9kb8lrse6u
Fetching download links for:  https://bitport.io/my-files/file/ptj13p1iqm
Fetching download links for:  https://bitport.io/my-files/file/995x0fuq2m
Fetching download links for:  https://bitport.io/my-files/file/5fs0k9ralt
Fetching download links for:  https://bitport.io/my-files/file/2qzt0c6a4q
Fetching download links for:  https://bitport.io/my-files/file/f1n7m7sq37
Fetching downl